In [3]:
import pandas as pd
from urllib.parse import urlparse
import hashlib
el_universal =  pd.read_csv('Web_Scrapper/eluniversal_2019_04_04_articles.csv')

In [4]:
#Obtención de host y origen
el_universal['host'] = el_universal['url'].apply(lambda url: urlparse(url).netloc)
el_universal['newspaper_uid'] = 'el_universal'

In [6]:
#Limpiado de datos
missing_title_mask = el_universal['title'].isna()
missing_titles = (el_universal[missing_title_mask]['url']
                  .str.extract(r'(?P<missing_titles>[^/]+)$')
                  .applymap(lambda title: title.split('-'))
                  .applymap(lambda title_word_list: ' '.join(title_word_list)))
el_universal.loc[missing_title_mask, 'title'] =  missing_titles.loc[:,'missing_titles']
uids = (el_universal
        .apply(lambda row: hashlib.md5(bytes(row['url'].encode())), axis=1)
        .apply(lambda hash_object: hash_object.hexdigest())
        )
el_universal['uid'] = uids
el_universal.set_index('uid', inplace=True)
stripped_body = (el_universal
                 .apply(lambda row: row['body'], axis=1)
                 .apply(lambda body: list(body))
                 .apply(lambda letters: list(map(lambda letter: letter.replace('\n',''), letters)))
                 .apply(lambda letters: ''.join(letters))
                )
el_universal['body'] = stripped_body
el_universal.head()

,body,title,url,host,newspaper_uid
uid,,,,,
d15f4cbfd390a45ee35dd33dd85cae5d,Un presidente con la mayor votación en la era ...,A lo ganso,https://www.eluniversal.com.mx/articulo/carlos...,www.eluniversal.com.mx,el_universal
1b2f1ae0f843aa1ff00b3ffc08b585eb,Debido a que presenta un desempeño presupuesta...,El Marqués aumenta su calificación crediticia,http://www.eluniversalqueretaro.mx/portada/el-...,www.eluniversalqueretaro.mx,el_universal
b0c37c2308b2371041ecb11307ebfc09,"CDMX.- La coordinadora del PES en el Senado, S...",Senadora Sasil de León sostiene que firma en d...,https://www.eluniversal.com.mx/nacion/senadora...,www.eluniversal.com.mx,el_universal
5f585ee4a8def807f2efe66b085f2fe4,¿Tu hijo tiene la ciudadanía estadounidense y ...,guia para tramitar el pasaporte estadounidense...,http://www.viveusa.mx/educacion/guia-para-tram...,www.viveusa.mx,el_universal
ef9e7ca72942f01240dd62198f200aeb,[email protected]Guadalajara.— La directiva de...,Las Chivas van por súper técnico,https://www.eluniversal.com.mx/universal-depor...,www.eluniversal.com.mx,el_universal


## Enriquecimiento

In [13]:
# Tokenizar el título y body
import nltk
from nltk.corpus import stopwords

#Librerías a descargar en el primer uso
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('spanish'))

def tokenize_columns(df, column_name):
    return (df
           .dropna()
           .apply(lambda row: nltk.word_tokenize(row[column_name]), axis=1)
           .apply(lambda tokens: list(filter(lambda token: token.isalpha(),tokens)))
           .apply(lambda tokens: list(map(lambda token: token.lower(),tokens)))
           .apply(lambda word_list: list(filter(lambda word: word not in stop_words, word_list)))
        .apply(lambda valid_word_list: len(valid_word_list))
           )

el_universal['n_tokens_title'] = tokenize_columns(el_universal,'title')
el_universal['n_tokens_body'] = tokenize_columns(el_universal,'body')

el_universal 

[nltk_data] Downloading package punkt to /home/ingkstr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ingkstr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,body,title,url,host,newspaper_uid,n_tokens_title,n_tokens_body
uid,,,,,,,
d15f4cbfd390a45ee35dd33dd85cae5d,Un presidente con la mayor votación en la era ...,A lo ganso,https://www.eluniversal.com.mx/articulo/carlos...,www.eluniversal.com.mx,el_universal,1,272
1b2f1ae0f843aa1ff00b3ffc08b585eb,Debido a que presenta un desempeño presupuesta...,El Marqués aumenta su calificación crediticia,http://www.eluniversalqueretaro.mx/portada/el-...,www.eluniversalqueretaro.mx,el_universal,4,235
b0c37c2308b2371041ecb11307ebfc09,"CDMX.- La coordinadora del PES en el Senado, S...",Senadora Sasil de León sostiene que firma en d...,https://www.eluniversal.com.mx/nacion/senadora...,www.eluniversal.com.mx,el_universal,7,129
5f585ee4a8def807f2efe66b085f2fe4,¿Tu hijo tiene la ciudadanía estadounidense y ...,guia para tramitar el pasaporte estadounidense...,http://www.viveusa.mx/educacion/guia-para-tram...,www.viveusa.mx,el_universal,6,263
ef9e7ca72942f01240dd62198f200aeb,[email protected]Guadalajara.— La directiva de...,Las Chivas van por súper técnico,https://www.eluniversal.com.mx/universal-depor...,www.eluniversal.com.mx,el_universal,4,133
14e379a73f4567cdc6301fa8135d297d,El único alimento que se le da al bebé hasta l...,Alimentos peligrosos para los bebés,https://www.eluniversal.com.mx/menu/alimentos-...,www.eluniversal.com.mx,el_universal,3,283
94ee861594399eef648f9779ad6b588f,La incapacidad como técnico de Bruno Marioni q...,¡Arriba Juárez! Enterrados los Pumas,https://www.eluniversal.com.mx/columna/gerardo...,www.eluniversal.com.mx,el_universal,3,240
98b5362895971245e295aacf0f084e8f,Las series televisivas fueron aprovechadas por...,Game of Thrones: la serie con más malware,https://www.eluniversal.com.mx/techbit/game-th...,www.eluniversal.com.mx,el_universal,5,115
6e73b605afc3de385942a575da621843,[email protected]Miles de abejas habitan dentr...,"Guardianes de las abejas, niñas y niños produc...",http://oaxaca.eluniversal.com.mx/especiales/01...,oaxaca.eluniversal.com.mx,el_universal,8,467
